In [2]:
print("Osei Tutu")

Osei Tutu


In [3]:
print("Hello World!")

Hello World!


In [4]:
%pwd

'c:\\Users\\Osei Tutu Dickson\\Desktop\\Gen AI\\mental-health-chatbot-gen-ai\\research'

In [12]:
import os


In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
#extract the data
def load_pdf_file(data):
    loader= DirectoryLoader(data, glob= '*.pdf', loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [ ]:
# import cryptography
# print(cryptography.__version__)


45.0.4


In [3]:
extracted_data = load_pdf_file(data='C:/Users/Osei Tutu Dickson/Desktop/Gen AI/mental-health-chatbot-gen-ai/Data/')

In [ ]:
# extracted_data

In [4]:
#split data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks =  text_splitter.split_documents(extracted_data)

    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print("The number of chunks are:", len(text_chunks))

The number of chunks are: 109


In [ ]:
text_chunks

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
# Download the embeddigs frmom hugging face
def download_from_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
    
    return embeddings

In [10]:
embeddings = download_from_hugging_face_embeddings()

C:\Users\Osei Tutu Dickson\AppData\Local\Temp\ipykernel_3440\2276683101.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Osei Tutu Dickson\miniconda3\envs\menbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
query_result = embeddings.embed_query('Hello World')
print('Length', len(query_result))

Length 384


In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [36]:
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mentalbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,  
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [38]:
# Embed each chunk and upsert the embeddings into your Pincode index
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings
)

In [39]:
# Embed each chunk and upsert the EXISTING embeddings into your Pincode index
docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)

In [40]:
docsearch

In [41]:
retriever = docsearch.as_retriever(search_type = 'similarity', search_kwargs={"k":3})

In [42]:
retrieved_docs = retriever.invoke("What is depression?")

In [43]:
retrieved_docs

[Document(id='7e10cbbd-0b7c-4295-ad4d-4688b3b8180a', metadata={'author': 'pooja zinta', 'creationdate': "D:20250327050044Z00'00'", 'creator': 'Canva', 'keywords': 'DAGUqRTE-90,BAFNEpJA3Lg,0', 'moddate': "D:20250327050044Z00'00'", 'page': 4.0, 'page_label': '5', 'producer': 'GPL Ghostscript 10.00.0', 'source': 'C:\\Users\\Osei Tutu Dickson\\Desktop\\Gen AI\\mental-health-chatbot-gen-ai\\Data\\ClarityintheChaos-updated.pdf', 'title': 'Clarity in the Chaos', 'total_pages': 42.0}, page_content="itself to the circumstances of each individual, often without our conscious\nawareness.\nIn today's fast-paced world, daily stress from work, relationships, and\nfinances builds up like a storm. Deeper stress from unresolved trauma or\nchronic anxiety lingers, impacting both mental and physical well-being."),
 Document(id='c8179be0-c05a-4a7d-8ff6-4582629c6fe2', metadata={'author': 'pooja zinta', 'creationdate': "D:20250327050044Z00'00'", 'creator': 'Canva', 'keywords': 'DAGUqRTE-90,BAFNEpJA3Lg,0', '